In [ ]:
import numpy as np
import pandas as pd

In [ ]:
filename = "/content/SectioLine_1.dat"

# Read the data
with open(filename, "r") as file:
    lines = file.readlines()

# Extract the number of sections
num_sections = int(lines[0].strip())
print(num_sections)

# Parse the sections
x_locations = []



106


In [ ]:
num_sections = int(lines[0])  # First line gives the number of sections
offset_data = {}
line_index = 1
for _ in range(num_sections):
    # Read x-location and number of points
    x_info = lines[line_index].strip().split(",")
    x_location = float(x_info[0])*0.3048
    num_points = int(x_info[1])

    # Read the (y, z) pairs and store them as (z, y)
    z_y_pairs = []
    for i in range(num_points):
        y, z = map(float, lines[line_index + 1 + i].strip().split(","))
        z_y_pairs.append((z*0.3048, y*0.3048))

    # Store the x-location with its associated (z, y) pairs
    offset_data[x_location] = z_y_pairs
    line_index += 1 + num_points

print(offset_data)

{-3.6009072000000004: [(9.11352, 0.0), (9.11352, 0.21244559999999998), (9.12114, 0.23804880000000003), (9.12114, 0.0)], -3.5999928000000003: [(9.049512, 0.0), (9.587514480000001, 3.08771544), (9.976408800000002, 4.16271456), (10.442448, 5.22890496), (10.5052368, 5.36454096), (10.925860799999999, 6.13233216), (10.999622400000002, 6.25821456), (11.4092736, 6.86171856), (11.496446400000002, 6.982419360000001), (11.870436000000002, 7.526487360000001), (11.9640096, 7.6337769600000005), (12.067336800000001, 7.741980960000001), (12.383109600000001, 8.044952160000001), (12.490704, 8.14370736), (12.9466848, 8.50794336), (13.0052064, 8.551529760000001), (13.432536, 8.831336160000001), (13.5581136, 8.905707360000001), (13.9421616, 9.11574504), (14.072616000000002, 9.18066744), (14.4709896, 9.36263304), (14.878202400000001, 9.52326264), (15.0150576, 9.57385944), (15.433243200000001, 9.71132424), (15.994380000000001, 9.768291360000001), (16.418966400000002, 9.767956080000001), (16.9999152, 9.680722

In [ ]:
max_y = float('-inf')
max_z = float('-inf')
for x_loc, z_y_pairs in offset_data.items():
    for z, y in z_y_pairs:
        if y > max_y:
            max_y = y
        if z > max_z:
            max_z = z

print(f"The maximum y coordinate is: {max_y}")
print(f"The maximum z coordinate is: {max_z}")

The maximum y coordinate is: 13.20000408
The maximum z coordinate is: 16.99997616


In [ ]:
x0=-11.814*0.3048
xl=537.534*0.3048
l=xl-x0
d=17
b=13.2*2

Offset Table

In [ ]:
# Input parameters
z_start = 0.0  # Start value for z
z_end = d      # End value for z
z_step = 0.1   # Step size for z

z_values = np.arange(z_start, z_end + z_step, z_step)  # Uniform z values
x_locations = sorted(offset_data.keys())  # Sorted x locations
columns = ['z'] + x_locations  # Column headers

# Populate the table
rows = []
for z in z_values:
    row = [z]
    for x_loc in x_locations:
        z_y_pairs = offset_data[x_loc]
        z_existing, y_existing = zip(*z_y_pairs)
        if z in z_existing:
            # Use the existing value if z is directly available
            y_val = y_existing[z_existing.index(z)]
        else:
            # Use linear interpolation if z is not directly available
            y_val = np.interp(z, z_existing, y_existing)
        row.append(round(y_val, 3))
    rows.append(row)

df = pd.DataFrame(rows, columns=columns)


print("Offset Table as DataFrame:")
print(df.head().to_string(index=False))

df.to_excel("offset.xlsx", index=False)

Offset Table as DataFrame:
  z  -3.600907  -3.599993  -3.000146  -1.800149  -1.199998  -0.0   0.600151   1.4478   3.130601   4.813402   4.924958   4.931664   5.234026   5.603138   5.971032   6.707124   8.179003   12.800015   12.80099   12.810012   14.397533   14.404238   20.800162   26.690117   32.580072   38.469722   44.359678   50.249633   56.139588   62.029543   67.919194   73.809149   79.699104   85.589059   91.479014   97.368665   103.25862   109.148575   115.03853   120.928486   126.818136   132.708091   138.598046   143.646449   144.798989   144.799995   147.01205   148.694851   149.59901   149.599985   153.743558   155.846983   157.529784   158.791961   159.633514   160.264145   160.579918   160.738109   160.817052   160.83595   160.856066   160.895386   160.921903   160.948116   161.001151   161.106002   161.316314   161.736938   162.157562   162.367874   162.578186   162.788803   162.999115   163.156697   163.261853   163.340796   163.38042   163.393526   163.399927   163.419

Sectional Area (Bonjean Curves)

In [ ]:
z_values = df.iloc[:, 0].to_numpy()
x_locations = df.columns[1:].astype(float)
sectional_area_table = []

for x in x_locations:
    y_values = df[x].to_numpy()
    cumulative_area = np.zeros_like(z_values)


    for i in range(len(z_values)):
        cumulative_area[i] = np.trapz(2 * y_values[:i+1], z_values[:i+1])

    sectional_area_table.append(cumulative_area)

sectional_area_df = pd.DataFrame(
    data=np.array(sectional_area_table).T,  # Transpose to align rows with z-values
    columns=x_locations,
    index=z_values
)


print(sectional_area_df.to_string(index=False))
sectional_area_df.to_excel("Sectional_Area.xlsx", index=False)

 -3.600907    -3.599993    -3.000146    -1.800149    -1.199998    -0.000000     0.600151     1.447800     3.130601     4.813402     4.924958     4.931664     5.234026     5.603138     5.971032     6.707124     8.179003     12.800015    12.800990    12.810012    14.397533    14.404238    20.800162    26.690117    32.580072    38.469722    44.359678    50.249633    56.139588    62.029543    67.919194    73.809149    79.699104    85.589059    91.479014    97.368665    103.258620   109.148575   115.038530   120.928486   126.818136   132.708091   138.598046   143.646449   144.798989   144.799995   147.012050   148.694851   149.599010   149.599985   153.743558   155.846983   157.529784   158.791961   159.633514   160.264145   160.579918   160.738109   160.817052   160.835950   160.856066   160.895386   160.921903   160.948116   161.001151   161.106002   161.316314   161.736938   162.157562   162.367874   162.578186   162.788803   162.999115   163.156697   163.261853   163.340796   163.380420

Waterplane Area

In [ ]:
Awp=np.zeros(len(df))

# Iterate over each row (each z-location)
for i, (z_value, row) in enumerate(df.iterrows()):
    x_values = df.columns.values[1:].astype(float)
    y_values = row.values[1:]


    Awp[i] = 2*np.trapz(y_values, x_values)

    print(f"Awp at z={row.values[0]}: {Awp[i]}")


Awp[0] = 0 if np.isnan(Awp[0]) else Awp[0]
Awp[-1] = 0 if np.isnan(Awp[-1]) else Awp[-1]

Awp at z=0.0: 0.0
Awp at z=0.1: 2658.76352029824
Awp at z=0.2: 2724.1626501448804
Awp at z=0.30000000000000004: 2776.3732598306406
Awp at z=0.4: 2824.9783476444
Awp at z=0.5: 2872.3596497421604
Awp at z=0.6000000000000001: 2914.99890020136
Awp at z=0.7000000000000001: 2953.26996097176
Awp at z=0.8: 2988.832180566
Awp at z=0.9: 3021.31009649904
Awp at z=1.0: 3050.62354149408
Awp at z=1.1: 3077.26593414264
Awp at z=1.2000000000000002: 3103.30671901296
Awp at z=1.3: 3127.7901815928
Awp at z=1.4000000000000001: 3148.82708497992
Awp at z=1.5: 3167.86759853712
Awp at z=1.6: 3185.3857211335203
Awp at z=1.7000000000000002: 3201.4208289057606
Awp at z=1.8: 3216.94647326208
Awp at z=1.9000000000000001: 3231.7070508122406
Awp at z=2.0: 3245.02520878344
Awp at z=2.1: 3257.04531920088
Awp at z=2.2: 3268.39922816688
Awp at z=2.3000000000000003: 3279.97538177856
Awp at z=2.4000000000000004: 3290.8430747294396
Awp at z=2.5: 3301.30786892808
Awp at z=2.6: 3310.697967618
Awp at z=2.7: 3319.76049491328
A

In [ ]:
z_values = df.iloc[:, 0].to_numpy()
print(z_values)

[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3
  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7
  2.8  2.9  3.   3.1  3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.   4.1
  4.2  4.3  4.4  4.5  4.6  4.7  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5
  5.6  5.7  5.8  5.9  6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9
  7.   7.1  7.2  7.3  7.4  7.5  7.6  7.7  7.8  7.9  8.   8.1  8.2  8.3
  8.4  8.5  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5  9.6  9.7
  9.8  9.9 10.  10.1 10.2 10.3 10.4 10.5 10.6 10.7 10.8 10.9 11.  11.1
 11.2 11.3 11.4 11.5 11.6 11.7 11.8 11.9 12.  12.1 12.2 12.3 12.4 12.5
 12.6 12.7 12.8 12.9 13.  13.1 13.2 13.3 13.4 13.5 13.6 13.7 13.8 13.9
 14.  14.1 14.2 14.3 14.4 14.5 14.6 14.7 14.8 14.9 15.  15.1 15.2 15.3
 15.4 15.5 15.6 15.7 15.8 15.9 16.  16.1 16.2 16.3 16.4 16.5 16.6 16.7
 16.8 16.9 17. ]


Underwater Volume

In [ ]:
disp=np.zeros(len(df))

for i in range(len(df)):
  disp[i]=np.trapz(Awp[:i+1],z_values[:i+1])
  print(f"Displacement at z={z_values[i]}: {disp[i]}")

disp[0] = 0 if np.isnan(disp[0]) else disp[0]
disp[-1] = 0 if np.isnan(disp[-1]) else disp[-1]

Displacement at z=0.0: 0.0
Displacement at z=0.1: 132.93817601491202
Displacement at z=0.2: 402.0844845370681
Displacement at z=0.30000000000000004: 677.1112800358442
Displacement at z=0.4: 957.1788604095962
Displacement at z=0.5: 1242.045760278924
Displacement at z=0.6000000000000001: 1531.4136877761002
Displacement at z=0.7000000000000001: 1824.8271308347562
Displacement at z=0.8: 2121.932237911644
Displacement at z=0.9: 2422.439351764896
Displacement at z=1.0: 2726.0360336645517
Displacement at z=1.1: 3032.430507446388
Displacement at z=1.2000000000000002: 3341.4591401041685
Displacement at z=1.3: 3653.013985134456
Displacement at z=1.4000000000000001: 3966.8448484630926
Displacement at z=1.5: 4282.679582638944
Displacement at z=1.6: 4600.342248622476
Displacement at z=1.7000000000000002: 4919.682576124441
Displacement at z=1.8: 5240.600941232832
Displacement at z=1.9000000000000001: 5563.033617436548
Displacement at z=2.0: 5886.870230416332
Displacement at z=2.1: 6211.973756815548


LCF

In [ ]:
lcf=np.zeros(len(df))

for i, (z_value, row) in enumerate(df.iterrows()):
    x_values = df.columns.values[1:].astype(float)
    y_values = row.values[1:]
    arr=2*x_values*y_values
    lcf[i]=np.trapz(arr,x_values)/Awp[i]
    print(f"Lcf at z={row.values[0]}: {lcf[i]}")

lcf[0] = 0 if np.isnan(lcf[0]) else lcf[0]
lcf[-1] = 0 if np.isnan(lcf[-1]) else lcf[-1]

Lcf at z=0.0: nan
Lcf at z=0.1: 85.45026399510472
Lcf at z=0.2: 85.24160140185927
Lcf at z=0.30000000000000004: 85.18649338070684
Lcf at z=0.4: 85.09919501657636
Lcf at z=0.5: 85.09182434230301
Lcf at z=0.6000000000000001: 85.06073887554028
Lcf at z=0.7000000000000001: 85.02341654275229
Lcf at z=0.8: 84.99721435450003
Lcf at z=0.9: 84.96280100934135
Lcf at z=1.0: 84.93589867512105
Lcf at z=1.1: 84.93409208592279
Lcf at z=1.2000000000000002: 84.92734409179243
Lcf at z=1.3: 84.93226963544602
Lcf at z=1.4000000000000001: 84.93956521189595
Lcf at z=1.5: 84.94742163826211
Lcf at z=1.6: 84.9575720061174
Lcf at z=1.7000000000000002: 84.96162662979646
Lcf at z=1.8: 84.95976306682972
Lcf at z=1.9000000000000001: 84.96862075077662
Lcf at z=2.0: 84.96289149493298
Lcf at z=2.1: 84.95941710693381
Lcf at z=2.2: 84.95502180212097
Lcf at z=2.3000000000000003: 84.95099129784795
Lcf at z=2.4000000000000004: 84.94330861584518
Lcf at z=2.5: 84.93569710687635
Lcf at z=2.6: 84.92455393296063
Lcf at z=2.7: 8

<ipython-input-11-b743b397bae7>:7: RuntimeWarning: invalid value encountered in scalar divide
  lcf[i]=np.trapz(arr,x_values)/Awp[i]


LCB

In [ ]:
lcb=np.zeros(len(df))

for i in range(len(df)):
  arr=lcf*Awp
  lcb[i]=np.trapz(arr[:i+1],z_values[:i+1])
  if disp[i] != 0:
    lcb[i]=lcb[i]/disp[i]
  else:
    lcb[i]=0
  print(f"lcb at z={z_values[i]}: {lcb[i]}")

lcb[0] = 0 if np.isnan(lcb[0]) else lcb[0]
lcb[-1] = 0 if np.isnan(lcb[-1]) else lcb[-1]

""" Using sectional area
lcb=np.zeros(len(df))
for i, (z_value, row) in enumerate(sectional_area_df.iterrows()):
    x_values = sectional_area_df.columns.values[1:].astype(float)
    Asec = row.values[1:]
    arr=x_values*Asec

    lcb[i] = np.trapz(arr, x_values)/disp[i]
    print(f"lcb at z={row.values[0]}: {lcb[i]}")

Gives same output
"""

lcb at z=0.0: 0.0
lcb at z=0.1: 85.45026399510472
lcb at z=0.2: 85.37957849680004
lcb at z=0.30000000000000004: 85.31223739433331
lcb at z=0.4: 85.26256264384293
lcb at z=0.5: 85.22424151880125
lcb at z=0.6000000000000001: 85.19626214958076
lcb at z=0.7000000000000001: 85.17145130810401
lcb at z=0.8: 85.1488786824906
lcb at z=0.9: 85.12791845543639
lcb at z=1.0: 85.10802416468118
lcb at z=1.1: 85.09054107102483
lcb at z=1.2000000000000002: 85.07575882511011
lcb at z=1.3: 85.06331183583767
lcb at z=1.4000000000000001: 85.05323418899937
lcb at z=1.5: 85.04514201105971
lcb at z=1.6: 85.03874564515651
lcb at z=1.7000000000000002: 85.03360852741584
lcb at z=1.8: 85.02914337880418
lcb at z=1.9000000000000001: 85.0253793879455
lcb at z=2.0: 85.02209918929252
lcb at z=2.1: 85.01890947166646
lcb at z=2.2: 85.01583081923515
lcb at z=2.3000000000000003: 85.01283460404248
lcb at z=2.4000000000000004: 85.00983466450525
lcb at z=2.5: 85.0067532636416
lcb at z=2.6: 85.00352759053435
lcb at z=2.7: 84.

VCB

In [ ]:
vcb=np.zeros(len(df))

for i in range(len(df)):
  arr=z_values*Awp
  vcb[i]=np.trapz(arr[:i+1],z_values[:i+1])
  if disp[i] != 0:
    vcb[i]=vcb[i]/disp[i]
  else:
    vcb[i]=0
  print(f"vcb at z={z_values[i]}: {vcb[i]}")


vcb at z=0.0: 0.0
vcb at z=0.1: 0.1
vcb at z=0.2: 0.13387550073315177
vcb at z=0.30000000000000004: 0.18123533121002416
vcb at z=0.4: 0.23074230124469744
vcb at z=0.5: 0.28112507792926783
vcb at z=0.6000000000000001: 0.33199988576441336
vcb at z=0.7000000000000001: 0.38318346610723164
vcb at z=0.8: 0.43458565943839833
vcb at z=0.9: 0.48615192846975397
vcb at z=1.0: 0.537837239043298
vcb at z=1.1: 0.5896077397126783
vcb at z=1.2000000000000002: 0.6414540585353496
vcb at z=1.3: 0.6933719120982066
vcb at z=1.4000000000000001: 0.7453332969883616
vcb at z=1.5: 0.7973114610832295
vcb at z=1.6: 0.8492956027643515
vcb at z=1.7000000000000002: 0.9012780809543135
vcb at z=1.8: 0.9532586205636902
vcb at z=1.9000000000000001: 1.0052402629847887
vcb at z=2.0: 1.0572171329909696
vcb at z=2.1: 1.1091792426079417
vcb at z=2.2: 1.161122729041385
vcb at z=2.3000000000000003: 1.2130547369278477
vcb at z=2.4000000000000004: 1.2649798880823455
vcb at z=2.5: 1.316897409802839
vcb at z=2.6: 1.368802947070239

TPC

In [ ]:
tpc=Awp*1.025/100
print(tpc)

[ 0.         27.25232608 27.92266716 28.45782591 28.95602806 29.44168641
 29.87873873 30.2710171  30.63552985 30.96842849 31.2688913  31.54197582
 31.80889387 32.05984936 32.27547762 32.47064289 32.65020364 32.8145635
 32.97370135 33.12499727 33.26150839 33.38471452 33.50109209 33.61974766
 33.73114152 33.83840566 33.93465417 34.02754507 34.11880397 34.20710876
 34.29060119 34.37082749 34.44919691 34.52685209 34.603081   34.67450713
 34.74399745 34.81159416 34.87991905 34.94925912 35.0167618  35.08184926
 35.14423536 35.21063999 35.27786005 35.34587594 35.4196733  35.49003281
 35.55859221 35.62461055 35.692666   35.76227306 35.83001095 35.89753283
 35.97007495 36.04725939 36.11986541 36.18830276 36.25815459 36.35074535
 36.44344728 36.52793191 36.61302891 36.69576568 36.78378686 36.88086686
 36.96600488 37.05606739 37.16226812 37.26387957 37.35587683 37.44236057
 37.53612163 37.65577639 37.76274796 37.85430577 37.93806592 38.03517108
 38.13666442 38.24468934 38.33838992 38.43480126 38.

Transverse Moment of Inertia (Ixx)

In [ ]:
Ixx=np.zeros(len(df))
bmt=np.zeros(len(df))

for i, (z_value, row) in enumerate(df.iterrows()):
    x_values = df.columns.values[1:].astype(float)
    y_values = row.values[1:]
    arr=2*(y_values**3)/3
    Ixx[i]=np.trapz(arr,x_values)

Ixx[0]=0 if np.isnan(Ixx[0]) else Ixx[0]
Ixx[-1]=0 if np.isnan(Ixx[-1]) else Ixx[0]

for i in range(len(Ixx)):
  print(f"Ixx at z={z_values[i]}: {Ixx[i]}")

Ixx at z=0.0: 0.0
Ixx at z=0.1: 106241.74859107973
Ixx at z=0.2: 111795.54895451557
Ixx at z=0.30000000000000004: 116345.13857009646
Ixx at z=0.4: 120843.06029514672
Ixx at z=0.5: 125177.75526191825
Ixx at z=0.6000000000000001: 129092.93311962343
Ixx at z=0.7000000000000001: 132583.1400388085
Ixx at z=0.8: 135901.7143324277
Ixx at z=0.9: 138825.29172860942
Ixx at z=1.0: 141445.15332817478
Ixx at z=1.1: 143836.75173903158
Ixx at z=1.2000000000000002: 146145.9180616974
Ixx at z=1.3: 148411.6937448643
Ixx at z=1.4000000000000001: 150307.40359391284
Ixx at z=1.5: 151962.379487162
Ixx at z=1.6: 153484.89755432907
Ixx at z=1.7000000000000002: 154829.6918046031
Ixx at z=1.8: 156136.20611446956
Ixx at z=1.9000000000000001: 157358.3067395928
Ixx at z=2.0: 158478.28502427493
Ixx at z=2.1: 159456.72013968386
Ixx at z=2.2: 160365.14905203017
Ixx at z=2.3000000000000003: 161300.84781125636
Ixx at z=2.4000000000000004: 162175.03755666577
Ixx at z=2.5: 163024.7649726845
Ixx at z=2.6: 163777.505742022

Transverse Metacentric Radius (BMt)

In [ ]:
bmt=Ixx/disp
bmt[0]=0 if np.isnan(bmt[0]) else bmt[0]
bmt[-1]=0 if np.isnan(bmt[-1]) else bmt[0]

for i in range(len(bmt)):
  print(f"bmt at z={z_values[i]}: {bmt[i]}")

bmt at z=0.0: 0.0
bmt at z=0.1: 799.1816329657051
bmt at z=0.2: 278.0399474583784
bmt at z=0.30000000000000004: 171.82572791275535
bmt at z=0.4: 126.24919468388127
bmt at z=0.5: 100.78352929106839
bmt at z=0.6000000000000001: 84.296577828745
bmt at z=0.7000000000000001: 72.65517801577136
bmt at z=0.8: 64.0462084058721
bmt at z=0.9: 57.30805670221079
bmt at z=1.0: 51.886751158616605
bmt at z=1.1: 47.432827029614806
bmt at z=1.2000000000000002: 43.73715551618008
bmt at z=1.3: 40.62719013636673
bmt at z=1.4000000000000001: 37.8909207029228
bmt at z=1.5: 35.48301397638632
bmt at z=1.6: 33.363799747788
bmt at z=1.7000000000000002: 31.471480000763112
bmt at z=1.8: 29.793569070675908
bmt at z=1.9000000000000001: 28.286420244949674
bmt at z=2.0: 26.920635043974293
bmt at z=2.1: 25.66925205772704
bmt at z=2.2: 24.527243153585015
bmt at z=2.3000000000000003: 23.49384284180647
bmt at z=2.4000000000000004: 22.542452319012792
bmt at z=2.5: 21.667837966050286
bmt at z=2.6: 20.851653179665938
bmt at 

<ipython-input-16-8691672b9630>:1: RuntimeWarning: invalid value encountered in divide
  bmt=Ixx/disp


Longitudinal Moment of Inertia about AP (Iyy_AP)

In [ ]:
Iyy_ap=np.zeros(len(df))

for i, (z_value, row) in enumerate(df.iterrows()):
    x_values = df.columns.values[1:].astype(float)
    y_values = row.values[1:]
    arr=2*(x_values**2)*(y_values)
    Iyy_ap[i]=np.trapz(arr,x_values)

Iyy_ap[0]=0 if np.isnan(Iyy_ap[0]) else Iyy_ap[0]
Iyy_ap[-1]=0 if np.isnan(Iyy_ap[-1]) else Iyy_ap[0]

for i in range(len(Iyy_ap)):
  print(f"Iyy_ap at z={z_values[i]}: {Iyy_ap[i]}")


Iyy_ap at z=0.0: 0.0
Iyy_ap at z=0.1: 22427304.435960352
Iyy_ap at z=0.2: 22952285.101033494
Iyy_ap at z=0.30000000000000004: 23420262.51997426
Iyy_ap at z=0.4: 23834923.160884205
Iyy_ap at z=0.5: 24281903.589786023
Iyy_ap at z=0.6000000000000001: 24674545.866234027
Iyy_ap at z=0.7000000000000001: 25025419.732812777
Iyy_ap at z=0.8: 25355435.235453587
Iyy_ap at z=0.9: 25656430.564769387
Iyy_ap at z=1.0: 25931996.54611457
Iyy_ap at z=1.1: 26194784.274836473
Iyy_ap at z=1.2000000000000002: 26451193.35822194
Iyy_ap at z=1.3: 26696486.335465968
Iyy_ap at z=1.4000000000000001: 26910962.93029459
Iyy_ap at z=1.5: 27108720.521814805
Iyy_ap at z=1.6: 27292984.03708316
Iyy_ap at z=1.7000000000000002: 27460895.701868374
Iyy_ap at z=1.8: 27620937.259947874
Iyy_ap at z=1.9000000000000001: 27779727.605133466
Iyy_ap at z=2.0: 27915486.795198135
Iyy_ap at z=2.1: 28039988.26835709
Iyy_ap at z=2.2: 28157897.432660453
Iyy_ap at z=2.3000000000000003: 28278608.85089744
Iyy_ap at z=2.4000000000000004: 28390

Longitudinal Moment of Inertia about LCF (Iyy_LCF)

In [ ]:
Iyy_lcf=Iyy_ap-(Awp*(lcf**2))

Iyy_lcf[0]=0 if np.isnan(Iyy_lcf[0]) else Iyy_lcf[0]
Iyy_lcf[-1]=0 if np.isnan(Iyy_lcf[-1]) else Iyy_lcf[-1]

for i in range(len(Iyy_lcf)):
  print(f"Iyy_lcf at z={z_values[i]}: {Iyy_lcf[i]}")

Iyy_lcf at z=0.0: 0.0
Iyy_lcf at z=0.1: 3013684.237899918
Iyy_lcf at z=0.2: 3158163.4834135137
Iyy_lcf at z=0.30000000000000004: 3272847.3660377227
Iyy_lcf at z=0.4: 3376788.760767691
Iyy_lcf at z=0.5: 3484242.970427595
Iyy_lcf at z=0.6000000000000001: 3583568.9198152944
Iyy_lcf at z=0.7000000000000001: 3676286.232121054
Iyy_lcf at z=0.8: 3762538.098246809
Iyy_lcf at z=0.9: 3846567.183410559
Iyy_lcf at z=1.0: 3924472.2556910105
Iyy_lcf at z=1.1: 3996003.2838572524
Iyy_lcf at z=1.2000000000000002: 4068116.437946245
Iyy_lcf at z=1.3: 4134201.80779827
Iyy_lcf at z=1.4000000000000001: 4193026.519254893
Iyy_lcf at z=1.5: 4249183.783915147
Iyy_lcf at z=1.6: 4301541.887171514
Iyy_lcf at z=1.7000000000000002: 4351509.88101197
Iyy_lcf at z=1.8: 4400498.592601981
Iyy_lcf at z=1.9000000000000001: 4447880.432855096
Iyy_lcf at z=2.0: 4490646.2590530105
Iyy_lcf at z=2.1: 4530301.127594009
Iyy_lcf at z=2.2: 4568697.537287444
Iyy_lcf at z=2.3000000000000003: 4608105.886742782
Iyy_lcf at z=2.4000000000

Longitudinal Metacentric Radius (BMl)

In [ ]:
bml=Iyy_lcf/disp

bml[0]=0 if np.isnan(bml[0]) else bml[0]
bml[-1]=0 if np.isnan(bml[-1]) else bml[-1]

for i in range(len(bml)):
  print(f"bml at z={z_values[i]}: {bml[i]}")

bml at z=0.0: 0.0
bml at z=0.1: 22669.817867531638
bml at z=0.2: 7854.477366988189
bml at z=0.30000000000000004: 4833.5442969794685
bml at z=0.4: 3527.85555598532
bml at z=0.5: 2805.245250903755
bml at z=0.6000000000000001: 2340.039760921367
bml at z=0.7000000000000001: 2014.5942429293887
bml at z=0.8: 1773.165999848238
bml at z=0.9: 1587.8899839569144
bml at z=1.0: 1439.6259650373831
bml at z=1.1: 1317.755930117683
bml at z=1.2000000000000002: 1217.4670607582002
bml at z=1.3: 1131.7235095792012
bml at z=1.4000000000000001: 1057.0180280379334
bml at z=1.5: 992.1787754424632
bml at z=1.6: 935.0482322178455
bml at z=1.7000000000000002: 884.5102938409376
bml at z=1.8: 839.693508807175
bml at z=1.9000000000000001: 799.5422531537179
bml at z=2.0: 762.8240615617278
bml at z=2.1: 729.285297225142
bml at z=2.2: 698.7650125644027
bml at z=2.3000000000000003: 671.1813172130339
bml at z=2.4000000000000004: 645.7104023854741
bml at z=2.5: 622.1910240892436
bml at z=2.6: 600.1424296850874
bml at z=

<ipython-input-19-066c45547c77>:1: RuntimeWarning: invalid value encountered in divide
  bml=Iyy_lcf/disp


MTC

In [ ]:
mtc=(1.025*disp*bml)/(100*l)

mtc[0]=0 if np.isnan(mtc[0]) else mtc[0]
mtc[-1]=0 if np.isnan(mtc[-1]) else mtc[-1]

for i in range(len(mtc)):
  print(f"mtc at z={z_values[i]}: {mtc[i]}")

mtc at z=0.0: 0.0
mtc at z=0.1: 184.48416787976163
mtc at z=0.2: 193.32853619455406
mtc at z=0.30000000000000004: 200.34896666602606
mtc at z=0.4: 206.71179043962167
mtc at z=0.5: 213.28965291273164
mtc at z=0.6000000000000001: 219.36993992197256
mtc at z=0.7000000000000001: 225.04567595087238
mtc at z=0.8: 230.32562650115796
mtc at z=0.9: 235.46950841790925
mtc at z=1.0: 240.238505863802
mtc at z=1.1: 244.6173130536451
mtc at z=1.2000000000000002: 249.0317553691292
mtc at z=1.3: 253.0772038978287
mtc at z=1.4000000000000001: 256.67818763971013
mtc at z=1.5: 260.1158823095638
mtc at z=1.6: 263.321009439188
mtc at z=1.7000000000000002: 266.3798248413952
mtc at z=1.8: 269.3786929973645
mtc at z=1.9000000000000001: 272.2791957314529
mtc at z=2.0: 274.8971268871438
mtc at z=2.1: 277.32461923460534
mtc at z=2.2: 279.6750744026623
mtc at z=2.3000000000000003: 282.08747596263765
mtc at z=2.4000000000000004: 284.36882622124415
mtc at z=2.5: 286.56496960017427
mtc at z=2.6: 288.55556014921285
m

In [ ]:
hydrostatics=pd.DataFrame({
    'z':z_values,
    'Waterplane Area':Awp,
    'Underwater volume':disp,
    'LCF':lcf,
    'LCB':lcb,
    'VCB':vcb,
    'BMt':bmt,
    'BMl':bml,
    'Ixx':Ixx,
    'Iyy_ap':Iyy_ap,
    'Iyy_lcf':Iyy_lcf,
    'TPC':tpc,
    'MTC':mtc
})

hydrostatics=hydrostatics.round(5)
hydrostatics=hydrostatics[:-1]
print(hydrostatics.to_string(index=False))

   z  Waterplane Area  Underwater volume      LCF      LCB     VCB       BMt         BMl          Ixx       Iyy_ap      Iyy_lcf      TPC       MTC
 0.0          0.00000            0.00000  0.00000  0.00000 0.00000   0.00000     0.00000      0.00000 0.000000e+00 0.000000e+00  0.00000   0.00000
 0.1       2658.76352          132.93818 85.45026 85.45026 0.10000 799.18163 22669.81787 106241.74859 2.242730e+07 3.013684e+06 27.25233 184.48417
 0.2       2724.16265          402.08448 85.24160 85.37958 0.13388 278.03995  7854.47737 111795.54895 2.295229e+07 3.158163e+06 27.92267 193.32854
 0.3       2776.37326          677.11128 85.18649 85.31224 0.18124 171.82573  4833.54430 116345.13857 2.342026e+07 3.272847e+06 28.45783 200.34897
 0.4       2824.97835          957.17886 85.09920 85.26256 0.23074 126.24919  3527.85556 120843.06030 2.383492e+07 3.376789e+06 28.95603 206.71179
 0.5       2872.35965         1242.04576 85.09182 85.22424 0.28113 100.78353  2805.24525 125177.75526 2.428190e+07 3.4

In [ ]:
hydrostatics.to_excel('Hydrostatics.xlsx', index=False)